<a href="https://colab.research.google.com/github/Buitragox/Crowdsourcing-Thesis/blob/main/notebooks/mv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Majority Voting Training

Base model with majority voting labels

Two models are trained
- Categorical crossentropy
- Focal loss

Dataset with oversampling was not used.

In [ ]:
import numpy as np
import zipfile

from matplotlib import pyplot as plt

from sklearn.metrics import classification_report

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!wget --no-check-certificate 'https://raw.githubusercontent.com/Buitragox/Crowdsourcing-Thesis/main/BCSS/utils.py' -O utils.py
!wget --no-check-certificate 'https://raw.githubusercontent.com/Buitragox/Crowdsourcing-Thesis/main/BCSS/grid_search.py' -O grid_search.py

--2024-05-29 16:27:02--  https://raw.githubusercontent.com/Buitragox/Crowdsourcing-Thesis/main/BCSS/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5405 (5.3K) [text/plain]
Saving to: ‘utils.py’

utils.py            100%[===================>]   5.28K  --.-KB/s    in 0s      

2024-05-29 16:27:03 (37.8 MB/s) - ‘utils.py’ saved [5405/5405]

--2024-05-29 16:27:03--  https://raw.githubusercontent.com/Buitragox/Crowdsourcing-Thesis/main/BCSS/grid_search.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5486 (5.4K) [text/plain

In [ ]:
from utils import load_mv_data
from grid_search import grid_search, show_results, save_to_json

In [ ]:
# pkl file with majority voting data
!wget --no-check-certificate 'https://github.com/Buitragox/Crowdsourcing-Thesis/raw/main/BCSS/pkl/mv.pkl' -O mv.pkl

--2024-05-29 16:27:24--  https://github.com/Buitragox/Crowdsourcing-Thesis/raw/main/BCSS/pkl/mv.pkl
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Buitragox/Crowdsourcing-Thesis/main/BCSS/pkl/mv.pkl [following]
--2024-05-29 16:27:24--  https://raw.githubusercontent.com/Buitragox/Crowdsourcing-Thesis/main/BCSS/pkl/mv.pkl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7261727 (6.9M) [application/octet-stream]
Saving to: ‘mv.pkl’

mv.pkl              100%[===================>]   6.92M  --.-KB/s    in 0.1s    

2024-05-29 16:27:25 (50.0 MB/s) - ‘mv.pkl’ saved [7261727/7261727]



In [ ]:
# Download TrainTestNpyInt.zip from google drive
!pip install gdown
!gdown 1XeVC0FOmv_V8jY31JP73yXqa4q27EWJS -O TrainTestNpyInt.zip

--2024-01-12 14:40:26--  https://docs.google.com/uc?export=download&confirm=&id=1XeVC0FOmv_V8jY31JP73yXqa4q27EWJS
Resolving docs.google.com (docs.google.com)... 74.125.197.113, 74.125.197.102, 74.125.197.139, ...
Connecting to docs.google.com (docs.google.com)|74.125.197.113|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1XeVC0FOmv_V8jY31JP73yXqa4q27EWJS&export=download [following]
--2024-01-12 14:40:27--  https://drive.usercontent.google.com/download?id=1XeVC0FOmv_V8jY31JP73yXqa4q27EWJS&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 74.125.20.132, 2607:f8b0:400e:c07::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|74.125.20.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2431 (2.4K) [text/html]
Saving to: ‘TrainTestNpyInt.zip’

TrainTestNpyInt.zip 100%[===================>]   2.37K  --.-KB/s    in 0s  

In [ ]:
# Extract zip file from drive or change the path to a local file
with zipfile.ZipFile("./drive/MyDrive/npy/TrainTestNpyInt.zip", 'r') as zip_ref:
    zip_ref.extractall("./TrainTestNpyInt")

In [ ]:
pkl_path = "./mv.pkl"
data_path = "./TrainTestNpyInt"
json_path = "./drive/MyDrive/Experiment results/mv"
K = 3

In [ ]:
def evaluate(model, X_test, Y_test):
    y_pred = model.predict(X_test)
    y_pred = np.argmax(y_pred, axis=1)
    report = classification_report(np.argmax(Y_test, axis=1), y_pred, output_dict=True)
    return report

## Categorical Cross-Entropy

In [ ]:
def build_model():
    input_layer = Input(shape=(512, ))

    model = Dense(128, activation='relu')(input_layer)
    model = Dropout(0.25)(model)
    model = Dense(64, activation='relu')(model)
    model = Dropout(0.25)(model)

    model = Dense(K, activation="softmax")(model)

    model = Model(input_layer, model, name="VGG16_MV")

    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
    return model

In [ ]:
X_train, labels, X_test, Y_test = load_mv_data(data_path, pkl_path)

history, report = grid_search(X_train, labels, X_test, Y_test, build_model, evaluate, repeat=10, epochs=20, validation_split=0.2)

Run #1
Epoch 1/10
7525/7525 [==============================] - 29s 4ms/step - loss: 0.5507 - accuracy: 0.7785 - val_loss: 0.4859 - val_accuracy: 0.8192
Epoch 2/10
7525/7525 [==============================] - 25s 3ms/step - loss: 0.4781 - accuracy: 0.8089 - val_loss: 0.4365 - val_accuracy: 0.8192
Epoch 3/10
7525/7525 [==============================] - 21s 3ms/step - loss: 0.4558 - accuracy: 0.8170 - val_loss: 0.4190 - val_accuracy: 0.8262
Epoch 4/10
7525/7525 [==============================] - 25s 3ms/step - loss: 0.4404 - accuracy: 0.8239 - val_loss: 0.4184 - val_accuracy: 0.8262
Epoch 5/10
7525/7525 [==============================] - 21s 3ms/step - loss: 0.4298 - accuracy: 0.8268 - val_loss: 0.4092 - val_accuracy: 0.8321
Epoch 6/10
7525/7525 [==============================] - 24s 3ms/step - loss: 0.4205 - accuracy: 0.8302 - val_loss: 0.3924 - val_accuracy: 0.8420
Epoch 7/10
7525/7525 [==============================] - 25s 3ms/step - loss: 0.4145 - accuracy: 0.8337 - val_loss: 0.4036 -

In [ ]:
show_results(history, report, K)

parameters = {}
	mean f1 scores: [0.870636569261079, 0.7424602150426194, 0.7421525437473369]
	std f1 scores: [0.009618207490091435, 0.013669248366279169, 0.025295654130973073]
	mean accuracy: 0.8206003666361136
	std accuracy: 0.010606562608308733


In [ ]:
save_to_json(json_path, history, report, 'history_mv_int_cce.json', 'report_mv_int_cce.json')

#CCE OS

NOTE: Oversampling was not used in the final analysis

In [ ]:
from imblearn.over_sampling import RandomOverSampler

In [ ]:
report_path = json_path + '/report_mv_int_cce_os.json'
history_path = json_path + '/history_mv_int_cce_os.json'

X_train, y_train, X_test, y_test = load_mv_data(data_path, pkl_path)

over_sampler = RandomOverSampler(random_state=42)
X_train_os, y_train_os = over_sampler.fit_resample(X_train, y_train)

In [ ]:
history, report = grid_search(X_train_os, y_train_os, X_test, y_test, build_model, evaluate, repeat=10, epochs=20, validation_split=0.2, report_path=report_path, history_path=history_path)

Run #1
Epoch 1/20
10865/10865 [==============================] - 43s 4ms/step - loss: 0.5449 - accuracy: 0.7801 - val_loss: 0.9538 - val_accuracy: 0.4428
Epoch 2/20
10865/10865 [==============================] - 35s 3ms/step - loss: 0.4748 - accuracy: 0.8078 - val_loss: 0.7284 - val_accuracy: 0.6742
Epoch 3/20
10865/10865 [==============================] - 34s 3ms/step - loss: 0.4548 - accuracy: 0.8158 - val_loss: 0.5587 - val_accuracy: 0.7743
Epoch 4/20
10865/10865 [==============================] - 37s 3ms/step - loss: 0.4426 - accuracy: 0.8220 - val_loss: 0.4055 - val_accuracy: 0.8260
Epoch 5/20
10865/10865 [==============================] - 33s 3ms/step - loss: 0.4318 - accuracy: 0.8241 - val_loss: 0.5758 - val_accuracy: 0.7477
Epoch 6/20
10865/10865 [==============================] - 35s 3ms/step - loss: 0.4242 - accuracy: 0.8273 - val_loss: 0.5350 - val_accuracy: 0.7686
Epoch 7/20
10865/10865 [==============================] - 35s 3ms/step - loss: 0.4190 - accuracy: 0.8293 - val_

In [ ]:
show_results(history, report, K)

parameters = {}
	mean f1 scores: [0.8446476550462363, 0.7112018081346209, 0.7250584071239089]
	std f1 scores: [0.01780267179224015, 0.017752672159123087, 0.05312968453526947]
	mean accuracy: 0.7919798350137489
	std accuracy: 0.017994373838409433


In [ ]:
save_to_json(history, report, history_path, report_path)

#Focal Loss


In [ ]:
from tensorflow.keras.losses import Loss

In [ ]:
class FocalLoss(Loss):
    def __init__(self, gamma=2.0, alpha=None, num_classes=None):
        super().__init__()
        self.gamma = gamma
        if num_classes is None:
            raise ValueError("The 'num_classes' parameter must be specified.")
        if alpha is None:
            alpha = [1 for _ in range(num_classes)]
        elif len(alpha) != num_classes:
            raise ValueError("The length of 'alpha' must be equal to 'num_classes'.")
        self.alpha = tf.convert_to_tensor(alpha, dtype=tf.float32)

    def call(self, y_true, y_pred):
        y_pred_clip = tf.clip_by_value(y_pred, clip_value_min=1e-4, clip_value_max=1-1e-4)
        cce = tf.keras.losses.CategoricalCrossentropy(reduction=tf.keras.losses.Reduction.NONE)
        entropy = cce(y_true, y_pred)

        weight = tf.math.pow(1.0 - y_pred_clip, self.gamma)
        focal = tf.reduce_sum(y_true * self.alpha * weight, axis=1)
        loss = focal * entropy

        return tf.reduce_mean(loss)

In [ ]:
# This function receives onehot encoded labels
# Give more importance to classes with less amount of samples in the dataset
def calculate_weights(y):
    y = np.argmax(y, axis=-1) # onehot to class value (0, 1, 2)
    counts = np.unique(y, return_counts=True)[1] # count labels per class
    weights = 1 / (counts / len(y)) # inverse of the percentaje of labels per class
    sum_weights = np.sum(weights)
    return weights / sum_weights

In [ ]:
def create_build_func(alpha: list[float]):
    def build_focal_loss(gamma=1.0):
        input_layer = Input(shape=(512, ))

        model = Dense(128, activation='relu')(input_layer)
        model = Dropout(0.25)(model)
        model = Dense(64, activation='relu')(model)
        model = Dropout(0.25)(model)

        model = Dense(K, activation="softmax")(model)

        model = Model(input_layer, model, name="VGG16_MV_FOCAL_LOSS")

        model.compile(optimizer="adam", loss=FocalLoss(gamma=gamma, alpha=alpha, num_classes=3), metrics=["accuracy"])
        return model

    return build_focal_loss

### Training with alpha

In [ ]:
X_train, labels, X_test, Y_test = load_mv_data(data_path, pkl_path)

In [ ]:
alpha = calculate_weights(labels)
print(alpha)
build_func = create_build_func(alpha)

[0.17102875 0.21392327 0.61504798]


In [ ]:
gamma = [1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0]
report_path = json_path + '/report_mv_int_focal.json'
history_path = json_path + '/history_mv_int_focal.json'

exp_history, exp_report = grid_search(X_train, labels, X_test, Y_test, build_func,
                                      evaluate, repeat=10, epochs=20, validation_split=0.2,
                                      report_path=report_path, history_path=history_path, gamma=gamma)

Finished experiments: {(2.0,), (2.5,), (5.0,), (4.0,), (1.0,), (1.5,), (3.5,), (3.0,)}
Experiment with parameters: {'gamma': 4.5}
Run #1
Epoch 1/20
7525/7525 [==============================] - 28s 4ms/step - loss: 0.0259 - accuracy: 0.7042 - val_loss: 0.0183 - val_accuracy: 0.7861
Epoch 2/20
7525/7525 [==============================] - 25s 3ms/step - loss: 0.0184 - accuracy: 0.7409 - val_loss: 0.0155 - val_accuracy: 0.7673
Epoch 3/20
7525/7525 [==============================] - 26s 4ms/step - loss: 0.0178 - accuracy: 0.7477 - val_loss: 0.0138 - val_accuracy: 0.8010
Epoch 4/20
7525/7525 [==============================] - 26s 3ms/step - loss: 0.0174 - accuracy: 0.7456 - val_loss: 0.0181 - val_accuracy: 0.6382
Epoch 5/20
7525/7525 [==============================] - 24s 3ms/step - loss: 0.0169 - accuracy: 0.7484 - val_loss: 0.0145 - val_accuracy: 0.8058
Epoch 6/20
7525/7525 [==============================] - 27s 4ms/step - loss: 0.0167 - accuracy: 0.7492 - val_loss: 0.0143 - val_accuracy: 

In [ ]:
show_results(exp_history, exp_report, K)

parameters = {'gamma': 1.0}
	mean f1 scores: [0.8411603572163026, 0.708225726936487, 0.6796639573186076]
	std f1 scores: [0.030799043784068017, 0.020545553037667072, 0.02948803885630939]
	mean accuracy: 0.7818056828597617
	std accuracy: 0.028706401942245295
parameters = {'gamma': 2.0}
	mean f1 scores: [0.8577773705301471, 0.711049542758571, 0.7038689136264422]
	std f1 scores: [0.02191886353121606, 0.023342237636032367, 0.021836805103992638]
	mean accuracy: 0.7982126489459213
	std accuracy: 0.02213622466546775
parameters = {'gamma': 2.5}
	mean f1 scores: [0.8216241734745765, 0.6976187041574846, 0.6723282488571836]
	std f1 scores: [0.03792005506784657, 0.030580918176924078, 0.04081207477657598]
	mean accuracy: 0.76487167736022
	std accuracy: 0.034377245266409964
parameters = {'gamma': 5.0}
	mean f1 scores: [0.7991351507380615, 0.6559009720513205, 0.6996828625759417]
	std f1 scores: [0.09812988857073406, 0.05465855018825096, 0.03265057766626842]
	mean accuracy: 0.7469065077910174
	std acc

### Training without alpha

In [ ]:
X_train, labels, X_test, Y_test = load_mv_data(data_path, pkl_path)
alpha = [1, 1, 1] # No alpha is the same as multiplying by 1
build_func = create_build_func(alpha)

gamma = [1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0]
report_path = json_path + '/report_mv_int_focal_noalpha.json'
history_path = json_path + '/history_mv_int_focal_noalpha.json'

exp_history, exp_report = grid_search(X_train, labels, X_test, Y_test, build_func,
                                      evaluate, repeat=10, epochs=20, validation_split=0.2,
                                      report_path=report_path, history_path=history_path, gamma=gamma)

Finished experiments: {(2.0,), (2.5,), (5.0,), (4.5,), (1.0,), (1.5,)}
Experiment with parameters: {'gamma': 3.0}
Run #1
Epoch 1/20
7525/7525 [==============================] - 36s 5ms/step - loss: 0.1563 - accuracy: 0.7455 - val_loss: 0.1018 - val_accuracy: 0.7993
Epoch 2/20
7525/7525 [==============================] - 35s 5ms/step - loss: 0.1145 - accuracy: 0.7788 - val_loss: 0.0984 - val_accuracy: 0.8015
Epoch 3/20
7525/7525 [==============================] - 35s 5ms/step - loss: 0.1100 - accuracy: 0.7839 - val_loss: 0.0997 - val_accuracy: 0.8093
Epoch 4/20
7525/7525 [==============================] - 32s 4ms/step - loss: 0.1055 - accuracy: 0.7921 - val_loss: 0.0916 - val_accuracy: 0.8103
Epoch 5/20
7525/7525 [==============================] - 32s 4ms/step - loss: 0.1030 - accuracy: 0.7939 - val_loss: 0.0917 - val_accuracy: 0.8066
Epoch 6/20
7525/7525 [==============================] - 32s 4ms/step - loss: 0.1017 - accuracy: 0.7949 - val_loss: 0.0903 - val_accuracy: 0.8131
Epoch 7/2

In [ ]:
show_results(exp_history, exp_report, K)

parameters = {'gamma': 1.0}
	mean f1 scores: [0.8720489995544985, 0.735010390026838, 0.7242394318290801]
	std f1 scores: [0.010600059365636974, 0.011255732751905539, 0.05318650510900476]
	mean accuracy: 0.8189505041246564
	std accuracy: 0.011010272532120093
parameters = {'gamma': 1.5}
	mean f1 scores: [0.8607005884843435, 0.7221105900106186, 0.711467052025877]
	std f1 scores: [0.016848781661231743, 0.015648673695522305, 0.041321620087278466]
	mean accuracy: 0.8059807516040329
	std accuracy: 0.017452878493319796
parameters = {'gamma': 2.0}
	mean f1 scores: [0.8490932563932034, 0.7141570657367197, 0.70934445907473]
	std f1 scores: [0.06507136443253886, 0.043315416276047916, 0.033220386698513474]
	mean accuracy: 0.7968148487626031
	std accuracy: 0.05592684830307111
parameters = {'gamma': 2.5}
	mean f1 scores: [0.8697185730724577, 0.7093011639520987, 0.701684663071329]
	std f1 scores: [0.010175386129575463, 0.039374018647466716, 0.03855817985858487]
	mean accuracy: 0.8102658111824015
	std 